In [1]:
import sys
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

from glob import glob

In [2]:
import pandas as pd
from scipy.stats import pearsonr

def get_cors(data, col, columns):
    cors=[]
    for i in columns:

        corr_coef, p_value = pearsonr(data[col],  data[i])
        cor_out =  pd.DataFrame([corr_coef, p_value]).T.assign(region = str(i))
        cor_out.columns = ['cor', 'p', 'region']

        cors.append(cor_out)

    all_cors = pd.concat(cors)
    
    return all_cors

def lin_mod(data, col, columns):
    metrics_all=[]
    for i in columns:
        x = np.array(data[i])
        y = np.array(data[col])
        # add a constant term to the x variable to fit the intercept
        X = sm.add_constant(x)
        # fit the linear regression model
        model = sm.OLS(y, X).fit()
        # extract the p-values and t-values for each coefficient
        p_values = model.pvalues[1]
        t_values = model.tvalues[1]
        metrics = pd.DataFrame([t_values, p_values]).T.assign(var=i)
        metrics.columns = ['t', 'p', 'var']
        metrics = metrics[['var', 't', 'p']]

        metrics_all.append(metrics)

    sig_p = pd.concat(metrics_all).sort_values('p', ascending=True)[['var', 'p']]
    
    return sig_p

In [3]:
sub_new_dist = pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/subj_net_distances.csv')

In [4]:
leiden_networks = (pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/leiden_network_glasser_regions.csv')
                   .iloc[:, :2])

leiden_networks.columns = ['parcel', 'network']   

vn_parcels = leiden_networks.query('network == "Visual Network"').parcel.to_list()
smn_parcels = leiden_networks.query('network == "Somatomotor Network"').parcel.to_list()
fpcn_parcels = leiden_networks.query('network == "Frontoparietal Network"').parcel.to_list()
dmn_parcels = leiden_networks.query('network == "Default Mode Network"').parcel.to_list()

In [5]:
net_search = (pd.read_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/analysis/leiden_network_glasser_regions.csv')
                   .iloc[:, 1:4].reset_index())
net_search['index'] = net_search['index'].astype(str)

def find_parcel(parcel):
    #return net_search.query('index.str.contains('+ '"'+ str(parcel) + '"'+')')
    return net_search.query('index == '+'"'+ str(parcel) + '"')

In [6]:
vector_path = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/leiden/subj_sm/sub*'
vector_sub_idx = sorted(glob(vector_path, recursive = True))
vector_sub_idx  = [s.replace('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/leiden/subj_sm/', '') for s in vector_sub_idx]

len(vector_sub_idx)

55

In [7]:
mat_path = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/sub*'
mat_sub_idx = sorted(glob(mat_path, recursive = True))
mat_sub_idx = [s.replace('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/', '') for s in mat_sub_idx]
mat_sub_idx = [s.replace('_sm_vector', '') for s in mat_sub_idx]

len(mat_sub_idx)

55

In [8]:
# find values in list1 that are not in list2
result = list(set(vector_sub_idx) - set(mat_sub_idx))
result 

[]

In [145]:
op_path = '/pl/active/banich/studies/wmem/fmri/operation_rsa/subj/subj_operation_sub*.mat'

op_sub_idx = sorted(glob(op_path, recursive = True))

op_sub_idx = [s.replace('/pl/active/banich/studies/wmem/fmri/operation_rsa/subj/subj_operation_', '') for s in op_sub_idx]
op_sub_idx = [s.replace('.mat', '') for s in op_sub_idx]
op_sub_idx = [s.replace('-', '') for s in op_sub_idx]

op_mat_sub_ids = pd.concat([pd.DataFrame(op_sub_idx, columns=['ops_sub']),
                            pd.DataFrame(vector_sub_idx, columns=['vector_sub'])],axis=1)

op_mat_sub_ids.columns = ['SubID', 'sub']
op_mat_sub_ids = op_mat_sub_ids.apply(lambda col: col.str.replace('sub', '', regex=False).astype(int))
op_mat_sub_ids.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/matched_subid.csv', index=False)

In [10]:
sub_paths = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/sub' #sub003_sm_vector
disp_paths = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/predictions/dispersion/'
dist_paths= '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/predictions/distance/'

import glob
sub_files = glob.glob(sub_paths+'*')
disp_files = glob.glob(disp_paths+'*')
dist_files = glob.glob(dist_paths+'*')

In [11]:
op_mat_sub_ids = op_mat_sub_ids.apply(lambda col: col.str.replace('sub', '', regex=False).astype(int))

In [18]:
z_data = pd.read_csv('/pl/active/banich/studies/wmem/fmri/mvpa/utaustin/data/group_mvpa_auc/ClearMem_Z_Average.csv')[['SubID']]
z_data['SubID'] = z_data['SubID'].astype(int)

In [19]:
z_data = pd.merge(z_data, op_mat_sub_ids, on ='SubID')

In [22]:
z_data = op_mat_sub_ids[[ 'SubID', 'sub']]

In [23]:
id_no_19 = op_mat_sub_ids.reset_index()#query('sub != 19').reset_index()
z_data_no_19 = z_data.reset_index()#.query('sub != 19').reset_index()

In [104]:
from glob import glob 

g1_path = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/*_sm_vector/*_sm_vector_g1_all_ops.csv'
g2_path = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/*_sm_vector/*_sm_vector_g2_all_ops.csv'
g3_path = '/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/subj/*_sm_vector/*_sm_vector_g3_all_ops.csv'

def sub_grads(paths, grad):
    g_dfs=[]
    for i in sorted(sorted(glob(paths, recursive = True))): 
        g_dfs.append((pd.read_csv(i).iloc[:, 2:]))
    sub_g = pd.concat(g_dfs).assign(gradient=grad)
    sub_g['sub'] = sub_g['sub'].str.replace('_sm_vector','').str.replace('sub','').astype(int)
    
    return sub_g

sub_g_list=[]
for i, j in zip([g1_path, g2_path, g3_path], range(1,4)):
    sub_g_list.append(sub_grads(i, j))

sub_gs=pd.concat(sub_g_list)

In [124]:
def new_grad_names(data, name):
    new_column_names = {col: name + str(col) for col in data.columns}
    data = data.rename(columns=new_column_names)
    return data

def long_wide_grads(data, sub, grad):
    s_g_main = data.query('sub =='+str(sub)+' and gradient =='+str(grad))[['maintain']].T
    s_g_replace = data.query('sub =='+str(sub)+' and gradient =='+str(grad))[['replace']].T
    s_g_suppress = data.query('sub =='+str(sub)+' and gradient =='+str(grad))[['suppress']].T
    s_g_clear = data.query('sub =='+str(sub)+' and gradient =='+str(grad))[['clear']].T

    s_g_main = new_grad_names(s_g_main, 'g'+str(grad)+'_main_').reset_index(drop=True)
    s_g_replace = new_grad_names(s_g_replace, 'g'+str(grad)+'_replace_').reset_index(drop=True)
    s_g_suppress = new_grad_names(s_g_suppress, 'g'+str(grad)+'_suppress_').reset_index(drop=True)
    s_g_clear = new_grad_names(s_g_clear, 'g'+str(grad)+'_clear_').reset_index(drop=True)

    wide_g = pd.concat([s_g_main, s_g_replace, s_g_suppress, s_g_clear], axis=1)
    
    return wide_g

all_sub_gs_wide = []
for i in sub_gs['sub'].unique():
    wide_sub_gs = []
    for j in range(1,4):
        wide_sub_gs.append(long_wide_grads(sub_gs, i, j))     
    final = pd.concat(wide_sub_gs, axis=1).assign(sub = i)
    all_sub_gs_wide.append(final)
    
wide_gs = pd.concat(all_sub_gs_wide)
print(wide_gs.shape)

wide_g_z_data = pd.merge(z_data_no_19, wide_gs, on ='sub').dropna()

def new_col_names(data, name):
    new_column_names = {col: name + str(col) for col in data.iloc[:, 2:].columns}
    data = data.rename(columns=new_column_names)
    return data

all_g_data = (wide_g_z_data
              .drop('index', axis=1)
              .filter(regex='^(?!.*(_accuracy|_evidence)).*$') 
             )

(55, 4321)


In [125]:
all_g_data

,SubID,sub,g1_main_0,g1_main_1,g1_main_2,g1_main_3,g1_main_4,g1_main_5,g1_main_6,g1_main_7,...,g3_clear_350,g3_clear_351,g3_clear_352,g3_clear_353,g3_clear_354,g3_clear_355,g3_clear_356,g3_clear_357,g3_clear_358,g3_clear_359
0,1,1,0.096287,-0.001073,-0.030895,0.132592,0.141616,0.140672,0.138880,-0.036815,...,0.011055,-0.007877,-0.023389,-0.021089,-0.024503,-0.013553,-0.013137,-0.020853,0.024105,-0.036172
1,4,2,0.100113,0.033883,0.004711,0.107606,0.111922,0.114756,0.087897,-0.052936,...,-0.015431,-0.011338,-0.035374,-0.027846,0.068788,-0.047175,0.017998,-0.012732,0.027671,0.023817
2,5,3,0.076635,-0.004018,0.091604,0.106068,0.113767,0.127786,0.114437,-0.054025,...,0.019268,-0.014238,-0.024485,-0.030425,0.009539,-0.000686,-0.044337,0.004507,-0.014533,-0.033267
3,6,4,0.119590,0.002613,0.036425,0.119000,0.129814,0.126561,0.113585,-0.043038,...,0.020049,-0.036406,-0.013684,-0.018184,-0.003479,0.013689,0.006492,-0.013862,0.062370,-0.046754
4,8,5,0.130980,0.013288,0.046193,0.136289,0.121332,0.128005,0.119203,-0.032997,...,-0.011425,-0.032874,-0.035987,-0.025094,0.049586,-0.042832,-0.027032,0.009725,0.072035,-0.006895
5,11,6,0.084146,0.001063,0.037563,0.091911,0.093080,0.079731,0.085263,-0.039516,...,-0.062342,-0.014736,0.000500,-0.021124,-0.022169,-0.046597,0.002934,-0.022177,0.011066,-0.045683
6,12,7,0.108205,0.038284,0.057636,0.119496,0.114681,0.119321,0.039835,-0.044172,...,-0.028808,0.000695,-0.027602,-0.029917,-0.023804,-0.019043,-0.019975,-0.006208,0.056698,0.018979
7,13,8,0.084766,0.024888,0.078711,0.111448,0.113668,0.107958,0.057308,-0.038913,...,0.027446,-0.007440,-0.008504,0.000405,-0.033046,-0.005035,-0.017004,0.025049,-0.021351,-0.028643
8,15,9,0.094775,0.013501,0.078921,0.105462,0.106466,0.090448,0.068054,0.000080,...,0.001565,-0.036749,-0.041495,-0.037012,0.016570,0.010450,0.004550,0.000461,-0.012628,-0.019163
9,17,10,0.083876,0.015714,0.047164,0.095720,0.107502,0.106787,0.104958,-0.032681,...,-0.005988,-0.005424,-0.028913,-0.031036,-0.017988,-0.019584,-0.007601,-0.026605,0.050154,-0.028351


In [126]:
def mah_cent_dist(data, sub, op):
    
    data_sub = (data
                .query('sub =='+str(sub))[['sub', 'gradient', op]].reset_index()
                .pivot(index=['index', 'sub'], columns='gradient', values=op)
               ).reset_index()
   
    from scipy.spatial.distance import mahalanobis

    # Generate a random data frame with 95 rows and 3 columns
    df = data_sub.iloc[:, 2:]

    # Calculate the mean vector for each column
    mu = df.mean()

    cov = df.cov()
    # Calculate the inverse of the covariance matrix
    inv_cov = np.linalg.inv(cov)

    # Calculate the pairwise Mahalanobis distance between each row and the mean vector
    distances = []
    for index, row in df.iterrows():
        distance = mahalanobis(row, mu, inv_cov)
        distances.append(distance)

    # Convert the distances list to a pandas series and add it as a new column to the data frame
    df['md'] = pd.DataFrame(distances, index=df.index)
    df = df.assign(op = op, sub=sub, parcel=data_sub['index'])
    df = df.assign(cent_mah_md = df['op'] +'_'+ df['parcel'].astype(str) + '_md')
    df = df[['cent_mah_md', 'sub', 'op', 'md']]
    return df


def mah_dist(data, sub, op, parcels, network):
    
    data_sub = (data
                .query('sub =='+str(sub))[['sub', 'gradient', op]].reset_index()
                .pivot(index=['index', 'sub'], columns='gradient', values=op)
               ).reset_index().iloc[parcels]
   
    from scipy.spatial.distance import mahalanobis

    # Generate a random data frame with 95 rows and 3 columns
    df = data_sub.iloc[:, 2:]

    # Calculate the mean vector for each column
    mu = df.mean()

    cov = df.cov()
    # Calculate the inverse of the covariance matrix
    inv_cov = np.linalg.inv(cov)

    # Calculate the pairwise Mahalanobis distance between each row and the mean vector
    distances = []
    for index, row in df.iterrows():
        distance = mahalanobis(row, mu, inv_cov)
        distances.append(distance)

    # Convert the distances list to a pandas series and add it as a new column to the data frame
    df['md'] = pd.DataFrame(distances, index=df.index)
    df['md'] = df['md']*-1
    df = df.assign(op = op, sub=sub, parcel=data_sub['index'], net=network)
    df = df.assign(net_mah_md = df['op'] +'_'+ df['parcel'].astype(str) + '_md_'+df['net'])
    df = df[['net_mah_md', 'parcel', 'sub', 'op', 'md']]
    return df

In [127]:
cent_mah_dist =[]
for i in sub_gs['sub'].unique():
    for j in ['maintain', 'replace', 'suppress', 'clear']:
            cent_mah_dist.append(mah_cent_dist(sub_gs, i, j))
            
cent_mah = (pd.concat(cent_mah_dist)
           .pivot(index='sub', columns='cent_mah_md', values='md').reset_index())
cent_mah_z = pd.merge(all_g_data[['sub', 'SubID']], cent_mah, on='sub')

In [128]:
net_mah_dist =[]
for i in sub_gs['sub'].unique():
    for j in ['maintain', 'replace', 'suppress', 'clear']:
        for k,l in zip([vn_parcels, smn_parcels, fpcn_parcels,dmn_parcels], 
                       ['vn', 'smn', 'fpcn','dmn']):
            net_mah_dist.append(mah_dist(sub_gs, i, j, k, l))
            
net_mah = (pd.concat(net_mah_dist)
           .pivot(index='sub', columns='net_mah_md', values='md').reset_index())

net_mah_z = pd.merge(all_g_data[['sub', 'SubID']], net_mah, on='sub')

In [129]:
mah_z = pd.merge(net_mah_z.drop(['sub'], axis=1),
                 cent_mah_z.drop(['sub'], axis=1), on ='SubID')

all_sub_data = pd.merge(all_g_data, mah_z, on='SubID')
#all_sub_data = pd.merge(all_sub_data, sub_new_dist, on='sub').drop(['sub'], axis=1)

In [130]:
def md_mean(data, op, parcels, net):
    means = (data.filter(regex='md')
             .filter(regex=op)
             .filter(regex=net)
             .sum(axis=1)
            )#.rename(columns=lambda x: x.replace(op+'_md_', '')).iloc[:, parcels].mean(axis=1)
    
    data[op+'_'+net+'_dispersion_md'] = means
    return data

for i in ['main', 'replace', 'suppress', 'clear']:
    for j,k in zip([vn_parcels, smn_parcels, fpcn_parcels, dmn_parcels], ['vn', 'smn', 'fpcn', 'dmn']):
        
        #nd_mean(all_sub_data, i, j, k)
        md_mean(all_sub_data, i, j, k)

In [131]:
all_sub_data.shape

(55, 7218)

In [141]:
disp_data = all_sub_data.filter(regex='SubID|dispersion_md')

disp_data.to_csv('/pl/active/banich/studies/wmem/fmri/operation_rsa/grp/gradients/dispersion_data/derosa_task_within_network_dispersion.csv')


In [142]:
disp_data.shape

(55, 17)

In [70]:
#pd.concat(sig_cols).query('p < .01').groupby('var').count().query('p > 2')

In [71]:
# 'RRS_depression' = .filter(regex='fpcn|dmn|vn|smn|'+target).filter(regex='^(?!.*(maintain_smn|replace_smn|suppress_smn)).*$') 

In [ ]:
#all_sub_data_with_rrs

In [ ]:
#!jupyter nbconvert --to script z_ave_predictions.ipynb